# Imports

In [141]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Read data

In [142]:
data = pd.read_csv('../data/train.csv')
data_to_predict = pd.read_csv('../data/test.csv')
data_ccrisk = pd.read_csv('../data/credit_risk_dataset.csv')

data_ccrisk.dropna(inplace=True)
data_ccrisk.drop_duplicates(inplace=True)

X_old = data.drop(['loan_status'], axis=1)
y_old = data['loan_status']

# Merge the dataframes
data_no_id = data.drop(['id'], axis=1)
merged_data = pd.concat([data_no_id, data_ccrisk], ignore_index=True)

def remove_col_prefix(dataframe, prefix):
	dataframe.columns = dataframe.columns.str.replace(prefix, '')
	return dataframe

merged_data = remove_col_prefix(merged_data, 'person_')
merged_data = remove_col_prefix(merged_data, 'loan_')


print(merged_data.isnull().sum())
print(merged_data.duplicated().sum())

age                    0
income                 0
home_ownership         0
emp_length             0
intent                 0
grade                  0
amnt                   0
int_rate               0
percent_income         0
cb_default_on_file     0
cb_cred_hist_length    0
status                 0
dtype: int64
0


# Outliers

In [143]:
from scipy.stats import zscore

numeric_columns = merged_data.select_dtypes(include=[np.number]).columns

def remove_outiers(dataframe, numeric_cols, z_threshold=4):
	dataframe = dataframe[(np.abs(zscore(dataframe[numeric_cols])) < z_threshold).all(axis=1)].copy()
	return dataframe

data_no_outliers = remove_outiers(merged_data, numeric_columns)

In [144]:
merged_data.shape, data_no_outliers.shape

((87146, 12), (85518, 12))

# Feature engineering

## Feature Engineering Pipeline

In [145]:
# Create income bins before calling the function
income_bins = pd.qcut(data_no_outliers['income'], q=5, retbins=True)[1]

def feature_engineering(dataframe, income_bins):

    # Map grades to numerical values
    grade_mapping = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7}
    dataframe['grade_numeric'] = dataframe['grade'].map(grade_mapping)

    # Map cb_default_on_file to binary values
    dataframe['cb_default_on_file'] = dataframe['cb_default_on_file'].map({'Y': 1, 'N': 0})
    
    # Feature engineering
    dataframe['total_income'] = (dataframe['emp_length'] + 1) * dataframe['income']
    dataframe['emp_length_to_age_ratio'] = dataframe['emp_length'] / dataframe['age']
    dataframe['income_to_age_ratio'] = dataframe['income'] / dataframe['age']
    dataframe['total_income_to_age_ratio'] = dataframe['total_income'] / dataframe['age']
    dataframe['income_category'] = pd.cut(dataframe['income'], bins=income_bins, labels=[1, 2, 3, 4, 5])
    dataframe['income_to_loan_ratio'] = dataframe['income'] / dataframe['amnt']
    dataframe['total_income_to_loan_ratio'] = dataframe['total_income'] / dataframe['amnt']
    dataframe['loan_income_age_interaction'] = dataframe['amnt'] / (dataframe['income'] * dataframe['age'])
    dataframe['income_grade_interaction'] = dataframe['income'] * dataframe['grade_numeric']
    dataframe['loan_grade_ratio'] = dataframe['amnt'] / dataframe['grade_numeric']
    dataframe['emp_to_cred_hist_ratio'] = dataframe['emp_length'] / (dataframe['cb_cred_hist_length'] + 1)
    dataframe['credit_behavior_score'] = (1 - dataframe['cb_default_on_file']) * dataframe['cb_cred_hist_length']
    dataframe['affordability_score'] = dataframe['income'] / (dataframe['amnt'] * (1 + dataframe['int_rate']/100))
    dataframe['interest_cost'] = dataframe['amnt'] * dataframe['int_rate'] / 100 
    dataframe['emp_length_loan_interaction'] = dataframe['emp_length'] * dataframe['amnt']

    # Lists of numeric and ordinal features
    numeric = [
        'total_income', 'emp_length_to_age_ratio', 'income_to_age_ratio', 
        'total_income_to_age_ratio', 'income_to_loan_ratio', 
        'total_income_to_loan_ratio', 'loan_income_age_interaction', 
        'income_grade_interaction', 'loan_grade_ratio', 'emp_to_cred_hist_ratio', 
        'credit_behavior_score', 'affordability_score', 'interest_cost', 
        'emp_length_loan_interaction'
    ]
    ordinal = ['income_category']

    return dataframe, numeric, ordinal


In [147]:
data_no_outliers, new_features_numeric, new_features_ordinal = feature_engineering(data_no_outliers, income_bins)
X = data_no_outliers.drop(['status'], axis=1)
y = data_no_outliers['status']

X

,age,income,home_ownership,emp_length,intent,grade,amnt,int_rate,percent_income,cb_default_on_file,...,income_to_loan_ratio,total_income_to_loan_ratio,loan_income_age_interaction,income_grade_interaction,loan_grade_ratio,emp_to_cred_hist_ratio,credit_behavior_score,affordability_score,interest_cost,emp_length_loan_interaction
0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,0,...,5.833333,5.833333,0.004633,70000,3000.000000,0.000000,14,5.232158,689.4,0.0
1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,0,...,14.000000,98.000000,0.003247,168000,1333.333333,2.000000,2,12.351125,534.0,24000.0
2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,0,...,4.800000,43.200000,0.007184,28800,6000.000000,0.727273,10,4.407713,534.0,48000.0
3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,0,...,5.833333,87.500000,0.005714,140000,6000.000000,2.333333,5,5.250053,1333.2,168000.0
4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,0,...,10.000000,30.000000,0.004545,60000,6000.000000,0.500000,3,9.352787,415.2,12000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87055,51,76960,MORTGAGE,0.0,PERSONAL,B,15000,9.99,0.19,0,...,5.130667,5.130667,0.003822,153920,7500.000000,0.000000,20,4.664666,1498.5,0.0
87122,51,80000,RENT,3.0,HOMEIMPROVEMENT,F,18000,18.78,0.23,1,...,4.444444,17.777778,0.004412,480000,3000.000000,0.142857,0,3.741745,3380.4,54000.0
87133,52,30000,RENT,6.0,VENTURE,C,5000,12.73,0.17,0,...,6.000000,42.000000,0.003205,90000,1666.666667,0.315789,18,5.322452,636.5,30000.0
87139,52,65004,RENT,4.0,PERSONAL,D,20000,15.58,0.31,1,...,3.250200,16.251000,0.005917,260016,5000.000000,0.200000,0,2.812078,3116.0,80000.0


In [148]:
all_data = feature_engineering(merged_data, income_bins)[0]

data_to_predict = remove_col_prefix(data_to_predict, 'person_')
data_to_predict = remove_col_prefix(data_to_predict, 'loan_')
data_to_predict = feature_engineering(data_to_predict, income_bins)[0]

In [149]:
column_names = all_data.columns
column_names

Index(['age', 'income', 'home_ownership', 'emp_length', 'intent', 'grade',
       'amnt', 'int_rate', 'percent_income', 'cb_default_on_file',
       'cb_cred_hist_length', 'status', 'grade_numeric', 'total_income',
       'emp_length_to_age_ratio', 'income_to_age_ratio',
       'total_income_to_age_ratio', 'income_category', 'income_to_loan_ratio',
       'total_income_to_loan_ratio', 'loan_income_age_interaction',
       'income_grade_interaction', 'loan_grade_ratio',
       'emp_to_cred_hist_ratio', 'credit_behavior_score',
       'affordability_score', 'interest_cost', 'emp_length_loan_interaction'],
      dtype='object')

In [150]:
X.dtypes

age                               int64
income                            int64
home_ownership                   object
emp_length                      float64
intent                           object
grade                            object
amnt                              int64
int_rate                        float64
percent_income                  float64
cb_default_on_file                int64
cb_cred_hist_length               int64
grade_numeric                     int64
total_income                    float64
emp_length_to_age_ratio         float64
income_to_age_ratio             float64
total_income_to_age_ratio       float64
income_category                category
income_to_loan_ratio            float64
total_income_to_loan_ratio      float64
loan_income_age_interaction     float64
income_grade_interaction          int64
loan_grade_ratio                float64
emp_to_cred_hist_ratio          float64
credit_behavior_score             int64
affordability_score             float64


# Preprocessing

In [151]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

In [152]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, shuffle=True, test_size=0.2, random_state=2024)

In [185]:
onehot_cols = ['home_ownership', 'intent', 'cb_default_on_file']
ordinal_cols = ['grade', 'income_category']
standardize_cols = [
    'age', 'income', 'emp_length', 'amnt', 'int_rate', 'percent_income', 
    'cb_cred_hist_length', 'total_income', 'emp_length_to_age_ratio', 
    'income_to_age_ratio', 'total_income_to_age_ratio', 'income_to_loan_ratio', 
    'total_income_to_loan_ratio', 'loan_income_age_interaction', 
    'income_grade_interaction', 'loan_grade_ratio', 'emp_to_cred_hist_ratio', 
    'credit_behavior_score', 'affordability_score', 'interest_cost', 
    'emp_length_loan_interaction', 'grade_numeric'
]
log_transform_cols = ['income', 'total_income', 'amnt', 'interest_cost']

In [186]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import SimpleImputer

def log_transform():
	return FunctionTransformer(np.log1p, validate=True)

# Imputers for missing values
ordinal_imputer = SimpleImputer(strategy='most_frequent')
onehot_imputer = SimpleImputer(strategy='most_frequent')
scaler_imputer = SimpleImputer(strategy='mean')
log_imputer = SimpleImputer(strategy='mean')

preprocessor = ColumnTransformer(
	transformers=[
		('ordinal', Pipeline([('imputer', ordinal_imputer), ('encoder', OrdinalEncoder())]), ordinal_cols),
		('onehot', Pipeline([('imputer', onehot_imputer), ('encoder', OneHotEncoder())]), onehot_cols),
		('scaler', Pipeline([('imputer', scaler_imputer), ('scaler', StandardScaler())]), standardize_cols),
		('log', Pipeline([('imputer', log_imputer), ('log', log_transform())]), log_transform_cols)
	], 
	remainder='passthrough',
	# force_int_remainder_cols=False
	)

preprocessor.fit(X_train)

X_train_prep = preprocessor.transform(X_train)
X_test_prep = preprocessor.transform(X_test)
data_prep = preprocessor.transform(all_data)
X_all_prep = preprocessor.transform(X)

In [187]:
standardize_cols_names = []
for name in standardize_cols:
	standardize_cols_names.append('std_' + name)

log_transform_cols_names = []
for name in log_transform_cols:
	log_transform_cols_names.append('log_' + name)

In [188]:
cols = preprocessor.transformers_[0][1]['encoder'].get_feature_names_out(ordinal_cols).tolist()
cols += preprocessor.transformers_[1][1]['encoder'].get_feature_names_out(onehot_cols).tolist()
cols += standardize_cols_names
cols += log_transform_cols_names

training_data_df = pd.DataFrame(X_train_prep, columns=cols)

In [174]:
training_plus_target = training_data_df.copy()
training_plus_target['status'] = y_train.values

In [175]:
training_plus_target.corr()['status'].sort_values(ascending=False)

status                             1.000000
grade                              0.390432
std_grade_numeric                  0.390432
std_percent_income                 0.380154
std_loan_income_age_interaction    0.363538
std_int_rate                       0.345783
std_interest_cost                  0.248579
home_ownership_RENT                0.235389
log_interest_cost                  0.220479
cb_default_on_file_1               0.186356
std_amnt                           0.133784
log_amnt                           0.105537
std_income_grade_interaction       0.096991
intent_DEBTCONSOLIDATION           0.062272
intent_MEDICAL                     0.050808
intent_HOMEIMPROVEMENT             0.031135
home_ownership_OTHER               0.003491
std_emp_length_loan_interaction   -0.002656
intent_PERSONAL                   -0.011013
std_cb_cred_hist_length           -0.013980
std_age                           -0.015303
intent_EDUCATION                  -0.056155
intent_VENTURE                  

In [181]:
# Shuffle columns with low correlation and recheck correlation

shuffled_col = training_plus_target['log_income'].copy()

permutation = np.random.permutation(shuffled_col.index)
shuffled_col = shuffled_col.reindex(permutation).reset_index(drop=True)

In [183]:
training_plus_target['control'] = shuffled_col

In [184]:
training_plus_target.corr()['status'].sort_values(ascending=False)

status                             1.000000
grade                              0.390432
std_grade_numeric                  0.390432
std_percent_income                 0.380154
std_loan_income_age_interaction    0.363538
std_int_rate                       0.345783
std_interest_cost                  0.248579
home_ownership_RENT                0.235389
log_interest_cost                  0.220479
cb_default_on_file_1               0.186356
std_amnt                           0.133784
log_amnt                           0.105537
std_income_grade_interaction       0.096991
intent_DEBTCONSOLIDATION           0.062272
intent_MEDICAL                     0.050808
intent_HOMEIMPROVEMENT             0.031135
home_ownership_OTHER               0.003491
control                            0.000086
std_emp_length_loan_interaction   -0.002656
intent_PERSONAL                   -0.011013
std_cb_cred_hist_length           -0.013980
std_age                           -0.015303
intent_EDUCATION                

# Models

In [76]:
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier

In [77]:
dtrain = xgb.DMatrix(X_train_prep, label=y_train)
dtest = xgb.DMatrix(X_test_prep, label=y_test)

In [78]:
xgb_params = [
	{    
		'objective': 'binary:logistic',
		'learning_rate'         : 0.03, 
		'n_estimators'          : 5000,
		'max_depth'             : 7, 
		'colsample_bytree'      : 0.75, 
		'colsample_bynode'      : 0.85,
		'colsample_bylevel'     : 0.45,                     
		'reg_alpha'             : 0.001, 
		'reg_lambda'            : 0.25,
		'enable_categorical'    : True,
		'random_state'          : 42,
	},
	{
	'objective': 'binary:logistic',
	'eval_metric': 'auc',
	'n_estimators' : 5000,
	'max_depth': 7,
	'eta': 0.08,
	'reg_lambda': 38.5,
	'subsample': 0.88,
	'colsample_bytree': 0.65,
	'enable_categorical': True,
	'min_child_weight': 5,
	},
]

lgb_params = [
	{
		'objective': 'binary', 
		'metric':    'auc',
		'learning_rate'       : 0.0325, 
		'n_estimators'        : 5_000,
		'max_depth'           : 7, 
		'num_leaves'          : 25, 
		'min_data_in_leaf'    : 20,
		'feature_fraction'    : 0.70, 
		'bagging_fraction'    : 0.88, 
		'bagging_freq'        : 6, 
		'lambda_l1'           : 0.001, 
		'lambda_l2'           : 0.1,
	},
	{
		'objective': 'binary', 
		'metric':    'auc',
		'learning_rate'       : 0.035,
		'data_sample_strategy': 'goss', 
		'n_estimators'        : 5_000,
		'max_depth'           : 7, 
		'num_leaves'          : 30, 
		'min_data_in_leaf'    : 30,
		'feature_fraction'    : 0.60, 
		'colsample_bytree'    : 0.65,
		'lambda_l1'           : 0.001, 
		'lambda_l2'           : 1.25,  
	},
	{
		'objective': 'binary',
		'metric': 'auc',
		'max_depth': -1,
		'verbosity': -1,
		'n_estimators': 1500,
		'max_bin': 1024,
		'boosting_type': 'gbdt',
		'colsample_bytree': 0.57,        
		'eta': 0.055,
		'reg_lambda': 10.8,
		'min_child_samples': 70,
	},
]

cat_params = [
	{
		'task_type'           : "CPU",
		'loss_function'       : 'Logloss',
		'eval_metric'         : "AUC",
		'bagging_temperature' : 0.25,
		'colsample_bylevel'   : 0.40,
		'iterations'          : 5_000,
		'learning_rate'       : 0.045,
		'max_depth'           : 7,
		'l2_leaf_reg'         : 0.80,
		'min_data_in_leaf'    : 30,
		'random_strength'     : 0.25,
		'random_state': 42,
	},
	{
		'iterations': 1500,
		'depth': 6,
		'eta': 0.3, 
		'reg_lambda': 41.0, 
		'loss_function': 'Logloss',
		'eval_metric': 'AUC',
		'random_state': 42,
		'min_data_in_leaf': 51,
		'early_stopping_rounds': 150,
		'verbose':200,
	},
	{
		'iterations': 3000,
		'depth': 7,
		'eta': 0.3, 
		'reg_lambda': 40.0, 
		'loss_function': 'Logloss',
		'eval_metric': 'AUC',
		'random_state': 42,
		'min_data_in_leaf': 51,
		'early_stopping_rounds': 300,
		'verbose':200,
	},
	{
		'task_type'           : "CPU",
		'loss_function'       : 'Logloss',
		'eval_metric'         : "AUC",
		'bagging_temperature' : 0.25,
		'colsample_bylevel'   : 0.40,
		'iterations'          : 7_000,
		'learning_rate'       : 0.035,
		'max_depth'           : 8,
		'l2_leaf_reg'         : 0.80,
		'min_data_in_leaf'    : 30,
		'random_strength'     : 0.25,
		'early_stopping_rounds': 200,
		'random_state': 42,
	},
]

In [79]:
from sklearn.ensemble import StackingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression

estimators = [
	('xgb0', xgb.XGBClassifier(**xgb_params[0])),
	('lgb0', lgb.LGBMClassifier(**lgb_params[0])),
	('lgb1', lgb.LGBMClassifier(**lgb_params[1])),
	('lgb2', lgb.LGBMClassifier(**lgb_params[2])),
	('cat0', CatBoostClassifier(**cat_params[0])),
	('cat1', CatBoostClassifier(**cat_params[1])),
	('cat2', CatBoostClassifier(**cat_params[2])),
	('cat3', CatBoostClassifier(**cat_params[3])),
]

In [80]:
for name, model in estimators:
	model.fit(X_train_prep, y_train)

[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] lambda_l1 is set=0.001, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.001
[LightGBM] [Warning] lambda_l2 is set=0.1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1
[LightGBM] [Warning] bagging_fraction is set=0.88, subsample=1.0 will be ignored. Current value: bagging_fraction=0.88
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] lambda_l1 is set=0.001, reg_alpha=0.0 will be ign

In [89]:
predictions_train = []
predictions_test = []
for name, model in estimators:
	predictions_train.append((name, model.predict_proba(X_train_prep)[:, 1]))
	predictions_test.append((name, model.predict_proba(X_test_prep)[:, 1]))

In [90]:
preds_train_df = pd.DataFrame(dict(predictions_train))
preds_test_df = pd.DataFrame(dict(predictions_test))

In [85]:
import optuna

In [92]:
from sklearn.metrics import roc_auc_score

def objective_final_model(trial):

	# Define the search space
	param = {
        'objective': 'binary',
        'metric': 'auc',
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),  # Adjusted num_leaves range
        'max_depth': trial.suggest_int('max_depth', 3, 10),      # Reduced max_depth
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 100),  # Reduced min_child_samples
        'min_child_weight': trial.suggest_float('min_child_weight', 1e-3, 10.0),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'min_gain_to_split': trial.suggest_float('min_gain_to_split', 0.0, 0.2),  # Added min_gain_to_split
    }
	
	# Define the model
	model = lgb.LGBMClassifier(**param)

	# Fit the model
	model.fit(preds_train_df, y_train,
			  eval_set=[(preds_test_df, y_test)],
			  eval_metric='auc',
			  callbacks=[lgb.early_stopping(stopping_rounds=100)],
			  )
	
	# Predict the test set
	y_pred_proba = model.predict_proba(preds_test_df)[:, 1]
	auc = roc_auc_score(y_test, y_pred_proba)

	return auc


study = optuna.create_study(direction='maximize', study_name='lgb_final_model')
study.optimize(objective_final_model, n_trials=250, timeout=300)

[I 2024-10-31 16:27:00,863] A new study created in memory with name: lgb_final_model


Training until validation scores don't improve for 100 rounds


[I 2024-10-31 16:27:01,516] Trial 0 finished with value: 0.9337021127642688 and parameters: {'learning_rate': 0.11335075031822978, 'num_leaves': 92, 'max_depth': 6, 'min_child_samples': 64, 'min_child_weight': 5.539188040420388, 'subsample': 0.7305733198574569, 'colsample_bytree': 0.5085683674413052, 'n_estimators': 357, 'min_gain_to_split': 0.067462951925701}. Best is trial 0 with value: 0.9337021127642688.


Early stopping, best iteration is:
[58]	valid_0's auc: 0.933702
Training until validation scores don't improve for 100 rounds


[I 2024-10-31 16:27:02,002] Trial 1 finished with value: 0.9340609529496166 and parameters: {'learning_rate': 0.2597934496195586, 'num_leaves': 44, 'max_depth': 9, 'min_child_samples': 42, 'min_child_weight': 6.770868592263297, 'subsample': 0.6448183505749767, 'colsample_bytree': 0.7179871085668887, 'n_estimators': 776, 'min_gain_to_split': 0.10372126330279954}. Best is trial 1 with value: 0.9340609529496166.


...

Training until validation scores don't improve for 100 rounds


[I 2024-10-31 16:31:54,323] Trial 248 finished with value: 0.9468795224805189 and parameters: {'learning_rate': 0.01658975733896057, 'num_leaves': 132, 'max_depth': 6, 'min_child_samples': 13, 'min_child_weight': 8.736320522412482, 'subsample': 0.7492198706793785, 'colsample_bytree': 0.5001994529430906, 'n_estimators': 780, 'min_gain_to_split': 0.04304890398322115}. Best is trial 234 with value: 0.9520154883895254.


Early stopping, best iteration is:
[368]	valid_0's auc: 0.94688
Training until validation scores don't improve for 100 rounds


[I 2024-10-31 16:31:55,887] Trial 249 finished with value: 0.9494591713615059 and parameters: {'learning_rate': 0.022697409460867096, 'num_leaves': 130, 'max_depth': 6, 'min_child_samples': 8, 'min_child_weight': 8.507573196903985, 'subsample': 0.732705203524037, 'colsample_bytree': 0.5225281433688027, 'n_estimators': 765, 'min_gain_to_split': 0.04585774754863016}. Best is trial 234 with value: 0.9520154883895254.


Early stopping, best iteration is:
[316]	valid_0's auc: 0.949459
Number of finished trials: 250
Best trial:
  Value:  0.9520154883895254
  Params: 
    learning_rate: 0.013069633249219299
    num_leaves: 135
    max_depth: 6
    min_child_samples: 15
    min_child_weight: 8.549825402280252
    subsample: 0.709645521107508
    colsample_bytree: 0.5020171365339475
    n_estimators: 708
    min_gain_to_split: 0.04982003917136881


In [93]:
best_params = study.best_params
best_score = study.best_value

print("Best score:", best_score)
print("Best params:")
for key, value in best_params.items():
	print(f"    {key}: {value}")



Best score: 0.9520154883895254
Best params:
    learning_rate: 0.013069633249219299
    num_leaves: 135
    max_depth: 6
    min_child_samples: 15
    min_child_weight: 8.549825402280252
    subsample: 0.709645521107508
    colsample_bytree: 0.5020171365339475
    n_estimators: 708
    min_gain_to_split: 0.04982003917136881


In [44]:
data_to_predict_prep = preprocessor.transform(data_to_predict)

y_probas_stacking = stacking_2.predict_proba(data_to_predict_prep)[:, 1]

submission = pd.DataFrame({'id': data_to_predict['id'], 'loan_status': y_probas_stacking})

submission.to_csv('../submissions/stacking_2_v1.csv', index=False)

In [46]:
submission

,id,loan_status
0,58645,0.995997
1,58646,0.046226
2,58647,0.522282
3,58648,0.012439
4,58649,0.067505
...,...,...
39093,97738,0.052030
39094,97739,0.007459
39095,97740,0.031909
39096,97741,0.115866
